<a href="https://colab.research.google.com/github/laryyssa/si_data-visualisation/blob/master/Projeto_Visualiza%C3%A7%C3%A3o_de_Dados_de_M%C3%BAsicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h1> Trabalho 02 - Visualização de Dados**

**Fonte dos Datasets** [aqui](https://zenodo.org/records/4778563)

**OBS**

Os datasets não estão nesse formato, eles foram tratados e modificados usando a biblioteca Pandas. Para analisar as mudanças e tratamentos feitos, acesse o [repositório no GitHub](https://github.com/laryyssa/si_data-visualisation)


----------------------------------------------
**Referência**

Gabriel P. Oliveira, Mariana O. Silva, Danilo B. Seufitelli, Anisio Lacerda, and Mirella M. Moro. Detecting Collaboration Profiles in Success-based Music Genre Networks. In Proceedings of the 21st International Society for Music Information Retrieval Conference (ISMIR 2020), 2020.

## Instalando Bibliotecas

In [7]:
import pandas as pd
import altair as alt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install uvicorn
!pip install python-multipart
!pip install fastapi
!pip install kaleido


In [10]:
# %cd "/content/drive/My Drive/Projetos/visualizacao-dados-musicas/data/requirements.txt"

# !pip install -r requirements.txt

[Errno 20] Not a directory: '/content/drive/My Drive/Projetos/visualizacao-dados-musicas/data/requirements.txt'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


**<h1> Trabalho 02**

**<h1>Questão 1</h1>**
## Qual a média de animação, danceabilidade, energia e streams das músicas tocadas no top 200 do Brasil durante os anos de 2017 a 2019?

In [8]:
q1_data = pd.read_csv('/content/drive/My Drive/Projetos/visualizacao-dados-musicas/data/q1_data.csv')
q1_data.head()

,Unnamed: 0,month,year,danceability,energy,streams
0,0,April,2017,0.682180,0.716642,666608907
1,1,April,2018,0.696298,0.697742,1365325611
2,2,April,2019,0.676173,0.683238,1638050327
3,3,August,2017,0.687942,0.708560,921262815
4,4,August,2018,0.695050,0.668515,1150392588


In [9]:
ordered_months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

dropdown_year = alt.binding_select(
    options=q1_data['year'].unique().tolist(),
    name='Escolha o ano '
)

dropdown_song = alt.binding_select(
    options=['energy', 'danceability', 'streams'],
    name='Categoria musical: '
)
ycol_param = alt.param(
    value='energy',
    bind=dropdown_song
)

grafico = alt.Chart(q1_data,  width=800, height=300).mark_line().encode(
    alt.X('month', type='nominal', title='Mês', sort=ordered_months),
    alt.Y('y', type='quantitative', title='Média categoria musical selecionada'),
    color=alt.Color('year:O', title='Ano', scale=alt.Scale( range=['red', 'green', 'blue'])),
).transform_calculate(
    y=f'datum[{ycol_param.name}]'

).add_params(
    ycol_param,
).properties(title='Média de Valência, danciabilidade e streams das musicas brasileiras durante os anos de 2017 a 2019')

circulos = grafico.mark_circle()
(grafico + circulos).properties(padding=20)

AttributeError: module 'altair' has no attribute 'param'

**<h2>Motivação**

A escolha foi feita para analisar as interações médias relacionadas à 'energia', 'danceabilidade' e 'número de streams' das músicas brasileiras, com o objetivo de observar a variação ao longo do ano. A representação visual dos dados foi realizada por meio de marcadores de linha e pontos para destacar a sequência temporal.

**<h2>Análise**

A análise revelou que, no período de 2017 a 2019, as músicas ouvidas no Brasil apresentaram pouca variação ao longo desse intervalo de tempo. Além disso, destaca-se que as médias de 'danceabilidade' e 'energia' mantiveram-se consistentemente elevadas.

Quanto aos streams, observou-se um padrão: nos meses de abril e julho, ocorreram picos na média de streamings, indicando um aumento ao longo do ano. Essa tendência pode estar relacionada a fatores como os períodos de férias e um aumento no número de ouvintes brasileiros no Spotify.

**<h1>Questão 2</h1>**
## Qual top 5 maiores artistas (aqueles que possuem mais streamings no top 200) de um determinado ano em um determinado país ou mundialmente (global) e quais seus gêneros musicais?

In [ ]:
q2_data = pd.read_csv('/content/drive/My Drive/Disciplinas/2023.2/Visualização de Dados/Trabalhos/Trabalho 02/analysis/q2_data.csv')
q2_data.columns

In [ ]:
dropdown_year = alt.binding_select(
    options= q2_data['year'].unique().tolist(),
    name='Ano: '
)

dropdown_country = alt.binding_select(
    options= q2_data['country'].unique().tolist(),
    name='País ou visão global: '
)

year_select = alt.selection_point(fields=['year'], bind=dropdown_year, value=2017)
country_select = alt.selection_point(fields=['country'], bind=dropdown_country, value='global')

def make_scale(country_select):
  y_scale = alt.Scale(
    domain=(0, 5000000000) if country_select == 'global' else (0, 100)
  )
  return y_scale

# y_scale = alt.Scale(domain=(0, 5000000000) if country_select == 'global' else (0, 10000000))
grafico = alt.Chart(q2_data, width=600, height=300).mark_bar(
    width=50,
    height=50
).encode(
    alt.X('artist_name', type='nominal', title='Artistas'),
    alt.Y('streams', type='quantitative', title='Número de Streams durante o ano'),
    tooltip=['image']
    # tooltip=alt.Tooltip(['image:N'], scale=alt.Scale(domain=[0, 1], range=[0, 1]))
).add_selection(
    year_select,
    country_select
).transform_filter(
    year_select & country_select
).properties(title='Top 5 Artistas do ano do país e ano selecionados')

generos = grafico.mark_text(align='center', baseline='middle', dy=-10).encode(
    text='genres'
)

(grafico + generos).properties(padding=20)

**<h2>Motivação**

Foi escolhido o canal visual da área das barras para representar o número de streams de artistas, tanto em nível nacional quanto mundial. A escolha de interações, como ano e visão global ou por país, foi feita pensando em análises individuais em períodos específicos e regiões determinadas.

A inclusão da imagem do artista ao passar o mouse serve primariamente como elemento decorativo, permitindo a visualização da foto do artista. Isso possibilita a observação de seu estilo, e, em alguns casos, sua etnia e gênero.

Os gêneros acima de cada barra indicam a categoria musical do respectivo artista, permitindo a identificação dos gêneros musicais mais populares em determinado ano em determinada região.

**<h2>Análise**

Desse gráfico podemos perceber diversas análises, com destaque para:


*   Visão global de 2019: Cada artista apresentado obteve um grande destaque, o que pode ser justificado pela pesquisa indicando que Ariana Grande, Billie Eilish e Post Malone lançaram álbuns de grande sucesso nesse ano. Além disso, eventos como o falecimento do rapper XXTENTATION no final de 2018 e o lançamento do filme da banda "Queen" no mesmo ano contribuíram para esse destaque.
*  Visões brasileiras: Ao longo dos anos, mesmo com a percepção comum de que o 'funk' e o 'pop' são os gêneros mais ouvidos, observa-se a presença marcante do gênero 'sertanejo' no Brasil. Isso pode ser explicado pela influência cultural histórica do agronegócio e sua interferência em vários ambientes, dentre um deles o musical.

**<h1>Questão 3</h1>**
## Análise de popularidade de uma música: o que faz ela ser mais popular? o tempo, a danceabilidade ou a energia?

In [ ]:
q3_data = pd.read_csv('/content/drive/My Drive/Disciplinas/2023.2/Visualização de Dados/Trabalhos/Trabalho 02/analysis/q3_data.csv')
q3_data.head()

In [ ]:
dropdown_xcol = alt.binding_select(
    options=['danceability', 'energy', 'duration_ms', 'valence'],
    name='Coluna X: '
)
xcol_param = alt.param(
    value='danceability',
    bind=dropdown_xcol
)

list_years = sorted(q3_data['interval'].unique().tolist())
input_dropdown = alt.binding_select(options=list_years, name='Ano de lançamento: ')
selection = alt.selection_single(fields=['interval'], bind=input_dropdown)
color = alt.condition(
    selection,
    alt.Color('interval:N', title='Ano de Lançamento', scale=alt.Scale(domain=list_years,  range=['red', 'green', 'blue', 'purple']), legend=None),
    alt.value('lightgray')
)

points = alt.Chart(q3_data, width=400, height=400).mark_circle().encode(
    alt.X('x', type='quantitative'),
    alt.Y('popularity', type='quantitative', title='Popularidade da Música'),
    color=color,
    tooltip = ['year_release', 'song_name']
).transform_calculate(
    x=f'datum[{xcol_param.name}]'
).add_params(
    xcol_param,
).add_selection(
    selection
).properties(title='Coluna X selecionada vs. Popularidade vs. Ano de Lançamento')

points

**<h2>Motivação**

Foi escolhido marcadores de pontos para representar cada música, com cores diferentes para indicar o ano de lançamento de cada uma. Essa escolha foi feita para destacar as características específicas de uma música em um determinado período de tempo.

**<h2>Análise**


*   Danceabilidade: Apesar da pouca quantidade de músicas dos anos mais antigos no dataset, é possível inferir que ao longo do tempo, as músicas se tornaram progressivamente mais dançantes. Em média, uma música com uma pontuação de danceabilidade entre 0.5 e 0.9 parece ter maior probabilidade de se tornar popular.
*   Energia: Assim como na danceabilidade, a energia das músicas aumentou ao longo dos anos. No entanto, observa-se que esse fator sozinho não determina a popularidade de uma música.
*   Duração da Música: é possível perceber que as músicas foram se tornando cada vez mais rápidas a partir dos anos 2000
*   Valencia: Este é o item mais distribuído entre os quatro analisados.

**Conclusão**: A popularidade de uma música parece depender de uma combinação de diversos valores. A observação da danceabilidade, energia, duração e valência ao longo do tempo revela mudanças significativas, indicando que são necessários múltiplos elementos para determinar se uma música vai ser aceita ou não pelo público








**<h1>Questão 4</h1>**
## Em que países o 10 gêneros mais populares do mundo são mais ouvidos? Houve uma mudança durante os anos?

In [ ]:
q4_data = pd.read_csv('/content/drive/My Drive/Disciplinas/2023.2/Visualização de Dados/Trabalhos/Trabalho 02/analysis/q4_data.csv')
q4_data.head()

In [ ]:
list_genres = q4_data['genres'].unique()

dropdown_genres = alt.binding_select(
    options=list_genres,
    name='Gênero Musical: '
)
x_param = alt.param(
    value='rap',
    bind=dropdown_genres
)
grafico = alt.Chart(q4_data, width=50, height=450).mark_bar().encode(
    alt.X('year:O', title='Ano'),
    alt.Y('sum(streams)', type='quantitative', title='Total de Streams'),
    alt.Color("year:N", title='Ano'),
    alt.Tooltip('sum(streams)')
).transform_calculate(
    x=f'datum[{x_param.name}]'
).add_selection(
    x_param
).transform_filter(
    alt.FieldEqualPredicate(field='genres', equal=x_param),
)

alt.layer(
    grafico.mark_bar(),
).facet(
    alt.Column("country:N", title='Quantidade de streams por um dos 10 gêneros musicais mais ouvidos no mundo em cada país'),
).configure_facet(spacing=10)


**<h2>Motivação**

Foram escolhidas a área das barras como canais visuais, dessa maneira é possível observar o volume de streamings de um determinado gênero escolhido nos países durante os anos de 2017-2019.  A inclusão da função de passar o mouse sobre as barras foi usada para fornecer dados precisos sobre o número exato de streamings.

**<h2>Análise**

A presença grande do rap e em outros gêneros nos Estados Unidos destaca-se de forma notável. Uma possível explicação para esse destaque constante dos EUA pode ser relacionada à origem do Spotify, criado nesse país. A fundação da plataforma nos Estados Unidos pode ter incentivado significativamente seu uso nessa região.

Além disso, chama a atenção a popularidade do sertanejo, aparecendo entre os 10 gêneros mais escutados globalmente, juntamente com gêneros como electro e pop, que também deixam sua marca aqui no país

É interessante notar, por exemplo, em gêneros como tropical house, que quando há uma diminuição nos streams nos EUA, essa tendência é refletida em outros países. Isso evidencia a influência significativa que os EUA exercem sobre as preferências musicais globais, destacando a potência de sua capacidade de influenciar tendências em escala internacional.